# 케라스 소개

## 설정

In [15]:
import tensorflow as tf
from tensorflow import keras

## 소개

Keras 및 TensorFlow API 개념에 대한 첫 번째 소개 역할을 합니다.

다음 방법을 배웁니다:

- TensorFlow의 텐서, 변수 및 그라디언트
- `Layer` 클래스를 서브 클래 싱하여 레이어 생성
- 저수준 훈련 루프 작성
- `add_loss()` 방법 을 통해 레이어에 의해 생성된 손실 추적
- 낮은 수준의 훈련 루프에서 메트릭 추적
- 컴파일된 실행 속도 `tf.function`
- 훈련 또는 추론 모드에서 레이어 실행
- 케라스 함수형 API

## 텐서

TensorFlow는 미분 프로그래밍을 위한 인프라 계층입니다. 그 핵심은 NumPy와 마찬가지로 N차원 배열(텐서)을 조작하기 위한 프레임워크입니다.

그러나 NumPy와 TensorFlow 사이에는 세 가지 주요 차이점이 있습니다:

- TensorFlow는 GPU 및 TPU와 같은 하드웨어 가속기를 활용할 수 있습니다.
- TensorFlow는 임의의 미분 가능한 텐서 표현식의 기울기를 자동으로 계산할 수 있습니다.
- TensorFlow 계산은 단일 머신의 많은 장치와 많은 수의 머신(각각 여러 장치가 있을 수 있음)에 분산될 수 있습니다.

TensorFlow의 핵심인 Tensor에 대해 살펴보겠습니다.

다음은 상수 텐서입니다.

In [16]:
x = tf.constant([[5, 2], 
                 [1, 3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)


다음 `.numpy()` 을 호출하여 값을 NumPy 배열로 얻을 수 있습니다.

In [17]:
x.numpy()

array([[5, 2],
       [1, 3]])

NumPy 배열과 매우 유사하며 다음과 같은 속성이 `dtype`, `shape`이 있습니다.

In [18]:
print("dtype:", x.dtype)
print("shape:", x.shape)

dtype: <dtype: 'int32'>
shape: (2, 2)


상수 텐서를 만드는 일반적인 방법은 `tf.ones` 와 `tf.zeros` 를 사용하는 것 입니다.

In [19]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


난수로 상수 텐서를 만들 수도 있습니다. 

In [36]:
x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)
print(x)
x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")
print(x)

tf.Tensor(
[[-1.2206277 -2.1888824]
 [ 1.0467587  1.8161092]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[4 6]
 [1 0]], shape=(2, 2), dtype=int32)


## 변수

변수는 변경 가능한 상태(예: 신경망의 가중치)를 저장하는 데 사용되는 특수 텐서입니다. `Variable` 일부 초기 값을 사용하여 생성 합니다.

In [41]:
initial_value = tf.random.normal(shape=(2, 2))
a = tf.Variable(initial_value)
print(a)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[-0.6608966 ,  0.11948887],
       [-1.2962437 ,  0.19271922]], dtype=float32)>


, 또는 다음  방법을 사용하여 `Variable` 의 값을 업데이트합니다 `.assign(value)`, `.assign_add(increment)`, or `.assign_sub(decrement)`:

In [44]:
new_value = tf.ones(shape=(2, 2))
a.assign(new_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j]

print(a)        

added_value = tf.ones(shape=(2, 2))
a.assign_add(added_value)
for i in range(2):
    for j in range(2):
        assert a[i, j] == new_value[i, j] + added_value[i, j]
        
print(a)        

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>


## 텐서플로우의 계산

NumPy를 사용한 적이 있다면 TensorFlow에서 수학을 수행하는 것이 매우 친숙해 보일 것입니다. 주요 차이점은 TensorFlow 코드가 GPU 및 TPU에서 실행될 수 있다는 것입니다.

In [46]:
a = tf.constant([[1,2],
                 [3,4]])
b = tf.constant([[1,1],
                 [2,2]])

c = a + b
print(c)
d = tf.square(c)
print(d)

tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 4  9]
 [25 36]], shape=(2, 2), dtype=int32)


## 그라디언트

NumPy의 또 다른 큰 차이점은 미분 가능한 표현식의 기울기를 자동으로 검색할 수 있다는 것입니다.

x 를 `tape.watch()` 열고 GradientTape를 통해 텐서를 "감시"하기 시작 하고 이 텐서를 입력으로 사용하여 미분 가능한 표현식을 작성하십시오.

In [47]:
x = tf.constant([1,3,5], dtype="float32")

with tf.GradientTape() as tape:
    tape.watch(x)  
    y = tf.square(x) + 2*x + 1    # x^2 + 2*x + 1  => 2*x + 2
    dy_dx = tape.gradient(y, x)
    print(dy_dx)  # [4, 8, 12]

tf.Tensor([ 4.  8. 12.], shape=(3,), dtype=float32)


기본적으로 변수는 자동으로 감시되므로 수동으로 변경할 필요가 없습니다.

In [48]:
x = tf.Variable([1,3,5], dtype="float32")

with tf.GradientTape() as tape:
    y = tf.square(x) + 2*x + 1  
    dy_dx = tape.gradient(y, x)
    print(dy_dx)

tf.Tensor([ 4.  8. 12.], shape=(3,), dtype=float32)


## 케라스 레이어

TensorFlow가 텐서, 변수 및 그라디언트를 처리하는 **미분 프로그래밍을 위한 인프라 계층**,
인 반면, Keras는 계층, 모델, 옵티마이저, 손실 함수, 메트릭 등을 다루는 **딥 러닝을 위한 사용자 인터페이스** 입니다.

Keras는 TensorFlow의 고급 API 역할을 합니다. Keras는 TensorFlow를 간단하고 생산적으로 만드는 것입니다.

`Layer` 클래스는 Keras의 기본 추상화입니다 . A `Layer`는 상태(가중치)와 일부 계산(call 메서드에 정의됨)을 캡슐화합니다.

간단한 레이어는 다음과 같습니다.

In [50]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        print('Linear.__init__()')
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        print('Linear.call()')
        return tf.matmul(inputs, self.w) + self.b   # (2,2)(2,4)+(4,) => (2,4)

Python 함수와 매우 유사한 인스턴스 를 사용 합니다.

In [52]:
linear_layer = Linear(units=4, input_dim=2)

y = linear_layer(tf.ones((2, 2)))
print(y)

Linear.__init__()
Linear.call()
tf.Tensor(
[[-0.11010438 -0.03130747  0.02373462 -0.06339031]
 [-0.11010438 -0.03130747  0.02373462 -0.06339031]], shape=(2, 4), dtype=float32)


에서 생성된 가중치 변수 는 다음 속성 `__init__`에서 자동으로 추적됩니다 `.weights` 

In [54]:
print(linear_layer.weights)
print(linear_layer.w, linear_layer.b)

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[-0.04670297,  0.03103117,  0.0286093 , -0.07406551],
       [-0.06340142, -0.06233865, -0.00487469,  0.01067521]],
      dtype=float32)>, <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[-0.04670297,  0.03103117,  0.0286093 , -0.07406551],
       [-0.06340142, -0.06233865, -0.00487469,  0.01067521]],
      dtype=float32)> <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>


## 레이어 가중치 생성

이 `self.add_weight()` 방법은 가중치 생성을 위한 바로 가기를 제공합니다.

In [55]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

linear_layer = Linear(4)

y = linear_layer(tf.ones((2, 2)))
print(y)

tf.Tensor(
[[ 0.1119335   0.03524943 -0.00179264 -0.0752756 ]
 [ 0.1119335   0.03524943 -0.00179264 -0.0752756 ]], shape=(2, 4), dtype=float32)


## 레이어 그라디언트

레이어를 내부에서 `GradientTape` 호출하여 레이어 가중치의 그라디언트를 자동으로 검색할 수 있습니다 . 이러한 그라디언트를 사용하여 수동으로 또는 최적화 개체를 사용하여 레이어의 가중치를 업데이트할 수 있습니다. 물론 필요한 경우 그라디언트를 사용하기 전에 수정할 수 있습니다.

In [56]:
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype("float32") / 255, y_train)
)
dataset = dataset.shuffle(buffer_size=1024).batch(64)

linear_layer = Linear(10)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

for step, (x, y) in enumerate(dataset):
    with tf.GradientTape() as tape:
        logits = linear_layer(x)
        loss = loss_fn(y, logits)

    gradients = tape.gradient(loss, linear_layer.trainable_weights)
    optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))

    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

Step: 0 Loss: 2.336012840270996
Step: 100 Loss: 2.277557373046875
Step: 200 Loss: 2.1292476654052734
Step: 300 Loss: 2.0603203773498535
Step: 400 Loss: 1.9213716983795166
Step: 500 Loss: 1.9477500915527344
Step: 600 Loss: 1.7986361980438232
Step: 700 Loss: 1.7613279819488525
Step: 800 Loss: 1.6687450408935547
Step: 900 Loss: 1.5995466709136963


## 훈련 가능한 가중치와 훈련 불가능한 가중치

레이어에 의해 생성된 가중치는 학습 가능하거나 학습 불가능할 수 있습니다. `trainable_weights`및 `non_trainable_weights` 각각 에 노출되어 있습니다. 다음은 훈련할 수 없는 가중치를 가진 레이어입니다.

In [61]:

class ComputeSum(keras.layers.Layer):

    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  

y = my_sum(x)
print(y.numpy())  

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


## 레이어를 소유한 레이어

레이어를 재귀적으로 중첩하여 더 큰 계산 블록을 생성할 수 있습니다. 각 레이어는 하위 레이어의 가중치를 추적합니다(학습 가능 및 학습 불가능 모두).

In [62]:
class MLP(keras.layers.Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

mlp = MLP()

y = mlp(tf.ones(shape=(3, 64)))

assert len(mlp.weights) == 6

위에서 수동으로 만든 MLP는 다음 기본 제공 옵션과 동일합니다.

In [63]:
mlp = keras.Sequential(
    [
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(32, activation=tf.nn.relu),
        keras.layers.Dense(10),
    ]
)